## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-30-18-55-09 +0000,cbc,Feds announce $7M for Hurricane Melissa relief...,https://www.cbc.ca/news/politics/hurricane-mel...
1,2025-10-30-18-54-31 +0000,bbc,Trump caps refugee admissions at record low - ...,https://www.bbc.com/news/articles/cy40jj71243o...
2,2025-10-30-18-52-17 +0000,startribune,"Marjorie Johnson, Minnesota’s ‘Blue Ribbon Bak...",https://www.startribune.com/marjorie-johnson-m...
3,2025-10-30-18-51-14 +0000,nyt,Trump Rule Could Ban Some Public Servants From...,https://www.nytimes.com/2025/10/30/business/tr...
4,2025-10-30-18-50-46 +0000,missionlocal,U.S. Navy found elevated plutonium in Bayview ...,https://missionlocal.org/2025/10/navy-elevated...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,62
58,new,26
137,china,19
236,xi,18
2,hurricane,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
111,2025-10-30-15-17-59 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...,155
227,2025-10-30-08-56-55 +0000,cbc,Trump sets new China tariff rate after 'amazin...,https://www.cbc.ca/news/world/us-china-trump-j...,147
271,2025-10-30-02-28-18 +0000,nypost,Trump says ‘we have a deal’ with China’s Xi Ji...,https://nypost.com/2025/10/29/us-news/trump-ch...,145
88,2025-10-30-16-06-07 +0000,bbc,Trump hails 'amazing' meeting with China's Xi ...,https://www.bbc.com/news/articles/crl25xl1gjpo...,144
138,2025-10-30-13-36-51 +0000,wapo,Trump cuts tariffs on China after ‘truly great...,https://www.washingtonpost.com/politics/2025/1...,135


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
111,155,2025-10-30-15-17-59 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...
271,65,2025-10-30-02-28-18 +0000,nypost,Trump says ‘we have a deal’ with China’s Xi Ji...,https://nypost.com/2025/10/29/us-news/trump-ch...
285,63,2025-10-30-01-32-00 +0000,wsj,Federal Reserve Chair Jerome Powell delivered ...,https://www.wsj.com/economy/central-banking/fe...
44,57,2025-10-30-17-31-09 +0000,nypost,"Hurricane Melissa leaves trail of death, destr...",https://nypost.com/2025/10/30/world-news/hurri...
270,51,2025-10-30-02-40-50 +0000,latimes,He said he was an immigration lawyer but raped...,https://www.latimes.com/california/story/2025-...
356,43,2025-10-29-21-39-39 +0000,nypost,"Putin envoy says US, Russia close to a ‘diplom...",https://nypost.com/2025/10/29/world-news/putin...
256,41,2025-10-30-05-52-08 +0000,nypost,"Nearly 37,000 VA employees furloughed or worki...",https://nypost.com/2025/10/30/us-news/departme...
201,40,2025-10-30-10-00-00 +0000,latimes,Warner Bros. Discovery is up for sale. Why CEO...,https://www.latimes.com/entertainment-arts/bus...
22,39,2025-10-30-18-12-38 +0000,nyt,"Volkswagen, Hit by Tariffs, Reports $1.5 Billi...",https://www.nytimes.com/2025/10/30/business/vo...
307,35,2025-10-29-23-40-00 +0000,wsj,Meta Shares Fall on Accelerating AI Spending D...,https://www.wsj.com/tech/metaplatforms-meta-q3...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
